In [152]:
# import

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 找不到element的例外
from selenium.common.exceptions import NoSuchElementException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 確認檔案是否已存在
import os

In [153]:
# Global Variable

website = 'website URL '
accont = 'your accont'
password = 'your password'
destDir = 'your destnition to download files'
curDestDir = destDir
driver = False
courses_Num = 0

In [154]:
with open('.credential/website', encoding='UTF-8') as wb:
    website = wb.readline()

with open('.credential/accont', encoding='UTF-8') as ac:
    accont = ac.readline()

with open('.credential/password', encoding='UTF-8') as pw:
    password = pw.readline()

with open('.credential/destDir', encoding='UTF-8') as dd:
    destDir = dd.readline()

In [155]:
# 啟動瀏覽器工具的選項
my_options = webdriver.EdgeOptions()
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")             #設定為正體中文

my_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,          #禁用下載的彈出視窗
    "download.directory_upgrade": True,             #可更新下載路徑
    #"safebrowsing.enabled": True                    #安全模式 (開了以後，資料結構 Data Structure \ CH2-2-2023 載不了，可能是因為檔名與連結標題不一致)
})

In [156]:
# functions for driver

def init():
    global driver
    if not driver:
        driver = webdriver.Edge(options=my_options)

# visit a website
def visit(URL):
    driver.get(URL)
    print('Going to', website)

# wait till some element shows up. Success with True, failed with False.
def wait_Element(CSS_target):
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, CSS_target)
            )
        )
        return True

    except TimeoutException:
        print('ERROR: Time Out!')
        return False

def login():
    # 等登入按鈕出現
    # 若先前已登入則返回
    if not wait_Element('button.btn.btn-primary.btn-lg.btn-block'):
        print('Failed: Aleady logged in!')
        return
    # 輸入帳密並登入
    driver.find_element(By.CSS_SELECTOR, 'input[placeholder=\"帳號\"]').send_keys(accont)
    driver.find_element(By.CSS_SELECTOR, 'input[placeholder=\"密碼 (區分大小寫)\"]').send_keys(password)
    driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-primary.btn-lg.btn-block').click()
    # 等詢問視窗pop up，點選保持登入
    wait_Element('.btn.btn-default.keepLoginBtn')
    driver.find_element(By.CSS_SELECTOR, '.btn.btn-default.keepLoginBtn').click()

def get_Courses_Num():
    global courses_Num
    wait_Element('tbody')
    courses_Num = len(driver.find_elements(By.CSS_SELECTOR, 'tbody tr'))
    print('Courses found:', courses_Num)

# check if the given file has been in curDestDir
def is_File_Exist(file_name):
    matching_Files = [file for file in os.listdir(curDestDir) if file.startswith(file_name)]
    return len(matching_Files) != 0

# download material file in the material page
def download_Material(file_name):
    global driver

    # avoid redownload file
    if is_File_Exist(file_name): 
        print('Failed to Download: File already exist:', file_name)
        return
    
    # wait till the page is loaded
    wait_Element('.navbar')

    WE_download = driver.find_elements(By.CSS_SELECTOR, '[title=\"下載\"], a.btn.btn-default.desktop-only') # 前者PPT，後者PDF

    # Some material cannot be downloaded
    if not WE_download:
        print('Failed: File is not downloadable:', file_name)
        return
    else:
        WE_download = WE_download[0]

    driver.execute_cdp_cmd('Browser.setDownloadBehavior', {'behavior':'allow', 'downloadPath': curDestDir})
    print('Downloaded:', curDestDir, '\\', file_name)
    WE_download.click()

# traverse materials in a course's materials page
def visit_Materials():
    wait_Element('tbody')

    # How many materials are in the course
    materials_Num = len(driver.find_elements(By.CSS_SELECTOR, 'tbody tr'))
    
    # If the course has no material
    try:
        driver.find_element(By.CSS_SELECTOR, 'tr#noData')
        print('-' * 60)
        print('Materials found: 0')
        return
    except NoSuchElementException:
        pass
   
    print('-' * 60)
    print('Materials found:', materials_Num)
    print('-' * 60)

    for i in range(1, materials_Num+1):
        # WebElement of ith material link
        WE_material_link = driver.find_element(By.CSS_SELECTOR, f'tbody tr:nth-child({i}) :nth-child(2) [href]')

        # Name and Link of ith material
        name_Material = WE_material_link.find_element(By.CSS_SELECTOR, 'span.text').text
        link_Material = WE_material_link.get_attribute('href')
        driver.get(link_Material)
        download_Material(name_Material)
        driver.back()

# traverse courses' materials page of a student
def visit_Courses():
    global courses_Num
    global driver
    global destDir
    global curDestDir

    for i in range(1, courses_Num+1):
        wait_Element('tbody')

        # WebElement of ith course
        WE_course = driver.find_element(By.CSS_SELECTOR, f'tbody tr:nth-child({i})')

        # Semester of ith course. Number as string.
        semester_Course = WE_course.find_element(By.CSS_SELECTOR, ':nth-child(1) .text-overflow').text

        # Name of ith course
        name_Course = WE_course.find_element(By.CSS_SELECTOR, ':nth-child(3) [href]').get_attribute('title')
        
        # Some character is not allowed in windows path so replace the shit
        for c in '<>:\"/\\|?*':
            if c in name_Course:
                name_Course = name_Course.replace(c, ' ')
                print('=' * 60)
                print('Warning: Course name has been modified to avoid illegal path name.')

        # Set the download directory
        curDestDir = destDir + '\\' + semester_Course + '\\' + name_Course
        if not os.path.exists(curDestDir): os.makedirs(curDestDir)

        # Link to ith course material 
        link_material = WE_course.find_element(By.CSS_SELECTOR, ':nth-child(3) [href]').get_attribute('href')
        link_material = link_material[0:-5] + 'material' + link_material[-6:]

        # go to ith course materials page and download each material
        driver.get(link_material)
        print('=' * 60)
        print("Enter Course:", name_Course)
        visit_Materials()
        driver.back()


In [157]:
# main

if __name__ == '__main__':
    init()
    visit(website)
    login()
    get_Courses_Num()
    visit_Courses()
    print('=' * 60)
    print('Download Finished!')

Going to https://ncueeclass.ncu.edu.tw/dashboard/historyCourse
Courses found: 34
Enter Course: 資料結構 Data Structure
------------------------------------------------------------
Materials found: 15
------------------------------------------------------------
Failed to Download: File already exist: CH6-1_2023-1 (3)
Failed to Download: File already exist: CH7-1-2023
Failed to Download: File already exist: CH6-2_2023-1
Failed to Download: File already exist: CH5-2_2023-1
Failed to Download: File already exist: CH5-1-2023
Failed to Download: File already exist: CH4-3_2023
Failed to Download: File already exist: CH4-2_2023
Failed to Download: File already exist: CH4-1-2023
Failed to Download: File already exist: CH2-4-2023
Failed to Download: File already exist: CH3-2_2023
Failed to Download: File already exist: CH3-1-2023
Downloaded: D:\NCU\教材\1121\資料結構 Data Structure \ CH2-2-2023
Failed to Download: File already exist: CH2-3-2023
Failed to Download: File already exist: CH2-1-2023
Failed to 

In [158]:
driver.quit()